# comercio_ext_indices.tb_nomenclatura_mercosul
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/NCM_delta`
## 📌 Descrição do arquivo

Tabela de referência da **NCM (Nomenclatura Comum do Mercosul)** com:

- Identificação do produto (NCM + descrição)
- Unidade estatística do produto
- Hierarquia SH6
- Classificações econômicas e setoriais (PPE, PPI, CUCI, ISIC, CGCE, SIIT…)
- Descrições multilíngues (PT/ES/EN)

Tipo: Dimensão Produto (hub) + bridges embutidos  
Uso: Enriquecimento e integração setorial (NCM → ISIC → CNAE)

|Coluna|Descrição técnica|Interpretação humana|
|---|---|---|
|`CO_NCM`|Código NCM do produto|Identificador do produto no COMEX|
|`CO_UNID`|Unidade estatística|Unidade oficial usada em `QT_ESTAT`|
|`CO_SH6`|Código SH 6 dígitos|Agrupamento internacional do produto (nível 6)|
|`CO_PPE`|Código PPE|Estágio da cadeia produtiva (primário/intermediário/final etc., conforme tabela PPE)|
|`CO_PPI`|Código PPI|Classificação complementar de produto (conforme tabela PPI)|
|`CO_FAT_AGREG`|Fator/agregação|Indicador técnico de agregação/agrupamento do produto|
|`CO_CUCI_ITEM`|Código CUCI item|Mapeamento do produto para classificação CUCI|
|`CO_CGCE_N3`|Código CGCE nível 3|Classificação econômica (CGCE) no nível N3|
|`CO_SIIT`|Código SIIT|Classificação complementar (SIIT)|
|`CO_ISIC_CLASSE`|Código ISIC classe|Atividade econômica associada ao produto (nível classe)|
|`CO_EXP_SUBSET`|Subconjunto exportação|Indicador de subset/segmento de exportação (uso analítico)|
|`NO_NCM_POR`|Descrição em português|Nome do produto em PT-BR|
|`NO_NCM_ESP`|Descrição em espanhol|Nome do produto em ES|
|`NO_NCM_ING`|Descrição em inglês|Nome do produto em EN|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
import org . apache . spark  . sql . functions . _
import org . apache . spark  . sql . types . _
import org . apache . spark  . sql . DataFrame
import org . apache . hadoop . fs  . FileSystem 
import org . apache . hadoop . fs  . Path 
import io  . delta  . tables . _

In [0]:

val bronzePath =     "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/ncm/"   
val silverPath =     "/mnt/silver/landingbeca2026jan/comercio_ext_indices/tb_nomenclatura_mercosul/"   
val silverTable =    "tb_nomenclatura_mercosul"
val fs = FileSystem.get(spark.sparkContext.hadoopConfiguration)



In [0]:

val silverSchema = StructType(
  Seq (
    StructField ( "CO_NCM" , StringType ,  nullable = false ) ,
    StructField ( "CO_UNID" , IntegerType ,  nullable = true ) ,
    StructField ( "CO_SH6" , StringType ,  nullable = true ) ,
    StructField ( "CO_PPE" , StringType ,  nullable = true ) ,
    StructField ( "CO_PPI" , StringType ,  nullable = true ) ,
    StructField ( "CO_FAT_AGREG" , StringType ,  nullable = true ) ,
    StructField ( "CO_CUCI_ITEM" , StringType ,  nullable = true ) ,
    StructField ( "CO_CGCE_N3" , StringType ,  nullable = true ) ,
    StructField ( "CO_SIIT" , StringType ,  nullable = true ) ,
    StructField ( "CO_ISIC_CLASSE" , StringType ,  nullable = true ) ,
    StructField ( "CO_EXP_SUBSET" , StringType ,  nullable = true ) ,
    StructField ( "NO_NCM_POR" , StringType ,  nullable = true ) ,
    StructField ( "NO_NCM_ESP" , StringType ,  nullable = true ) ,
    StructField ( "NO_NCM_ING" , StringType ,  nullable = true ) ,
    StructField ( "TS_REF" , TimestampType ,  nullable = false ) ,
    StructField ( "NM_ORIGEM" , StringType ,  nullable = false ) 
  )
)



## Extração
> #### **saprk.read**

In [0]:
val dfBronzeRaw = spark . read . format ( "delta" ) . load ( bronzePath )

In [0]:
val dfNormalized = dfBronzeRaw
  .withColumn("CO_NCM", trim(upper(col("CO_NCM").cast(StringType))))
  .withColumn("CO_UNID", col("CO_UNID").cast(IntegerType))
  .withColumn("CO_SH6", trim(upper(col("CO_SH6").cast(StringType))))
  .withColumn("CO_PPE", trim(upper(col("CO_PPE").cast(StringType))))
  .withColumn("CO_PPI", trim(upper(col("CO_PPI").cast(StringType))))
  .withColumn("CO_FAT_AGREG", trim(upper(col("CO_FAT_AGREG").cast(StringType))))
  .withColumn("CO_CUCI_ITEM", trim(upper(col("CO_CUCI_ITEM").cast(StringType))))
  .withColumn("CO_CGCE_N3", trim(upper(col("CO_CGCE_N3").cast(StringType))))
  .withColumn("CO_SIIT", trim(upper(col("CO_SIIT").cast(StringType))))
  .withColumn("CO_ISIC_CLASSE", trim(upper(col("CO_ISIC_CLASSE").cast(StringType))))
  .withColumn("CO_EXP_SUBSET", trim(upper(col("CO_EXP_SUBSET").cast(StringType))))
  .withColumn("NO_NCM_POR", col("NO_NCM_POR").cast(StringType))
  .withColumn("NO_NCM_ESP", col("NO_NCM_ESP").cast(StringType))
  .withColumn("NO_NCM_ING", col("NO_NCM_ING").cast(StringType))

In [0]:
val dfWithDefaults = dfNormalized
  .withColumn("TS_REF", current_timestamp())
  .withColumn("NM_ORIGEM", lit("/landingbeca2026jan/balancacomercial/ncm"))

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:

val dfValid = dfWithDefaults . filter ( 
  col("CO_NCM").isNotNull
)



In [0]:
val dfDedup = dfValid . dropDuplicates (
  Seq (
    "CO_NCM"
  )
)


In [0]:
val dfSilver = spark.createDataFrame(dfDedup.rdd, silverSchema.fieldNames.foldLeft(dfDedup.schema)((schema, name) => schema))

##Carga
> #### **overwrite**

In [0]:
spark.conf.set("spark.databricks.optimizer.dynamicPartitionPruning", "true")
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

val silverExists = fs.exists(new Path(silverPath))

if (!silverExists) {
  dfSilver
    .write
    .format("delta")
    .mode("overwrite")
    .save(silverPath)
}